In [54]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import string
import nltk
import regex
import dateparser
from nltk.corpus import stopwords
nltk.download('stopwords')

pd.set_option('display.max_columns', None) 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mossyhead/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Загрузка DF

In [74]:
csv_path = '/Users/mossyhead/ds_bootcamp/GameExplorer/steam_data/csv_files/raw_data.csv'
df = pd.read_csv(csv_path, dtype={'required_age': 'str'})
df.columns, len(df)

(Index(['Unnamed: 0', 'steam_appid', 'type', 'name', 'required_age',
        'platforms', 'categories', 'genres', 'detailed_description',
        'short_description', 'about_the_game', 'min_req', 'rec_req', 'is_free',
        'developers', 'publishers', 'release_date', 'metacritic'],
       dtype='object'),
 144786)

In [75]:
steam_apps = pd.read_csv('/Users/mossyhead/ds_bootcamp/GameExplorer/steam_data/csv_files/steam_apps.csv')
steam_apps.columns, len(steam_apps)

(Index(['Unnamed: 0', 'appid', 'name', 'got_info'], dtype='object'), 203750)

In [76]:
df.isna().sum()

Unnamed: 0                  0
steam_appid                 0
type                        0
name                       21
required_age                0
platforms                   0
categories               4860
genres                   4536
detailed_description     4408
short_description        4411
about_the_game           4410
min_req                     2
rec_req                     2
is_free                     0
developers                  9
publishers              14651
release_date              218
metacritic                  0
dtype: int64

# Основные функции

In [77]:
# Поиск и замена пропущенных значений в столбце 'name'

missing_names = df[pd.isna(df['name'])]
for index, row in missing_names.iterrows():
    appid = row['steam_appid']

    matching_row = steam_apps[steam_apps['appid'] == appid]
    if not matching_row.empty:
        matching_name = matching_row.iloc[0]['name']
        df.loc[index, 'name'] = matching_name

In [78]:
# Удаление строк, в которых нет описания
df = df.drop(df.loc[df['detailed_description'].isna()].index)
df = df.drop(df.loc[df['genres'].isna()].index)
df = df.drop(df.loc[df['categories'].isna()].index)
df = df.drop(df.loc[df['min_req'].isna()].index)

df.isna().sum()

Unnamed: 0                  0
steam_appid                 0
type                        0
name                        0
required_age                0
platforms                   0
categories                  0
genres                      0
detailed_description        0
short_description          14
about_the_game              2
min_req                     0
rec_req                     0
is_free                     0
developers                  6
publishers              10221
release_date              157
metacritic                  0
dtype: int64

In [79]:
# Функция для конвертации даты в формат "месяц год"

def convert_release_date(date_str):
    if isinstance(date_str, str):
        parsed_date = dateparser.parse(date_str)
        if parsed_date:
            return parsed_date.strftime('%b %Y')
        else:
            return date_str
    else:
        return date_str

In [80]:
# OneHotEncoding для столбцов 'platforms', 'categories', 'genres'

one_hot_columns = ['platforms', 'categories', 'genres']
def str_to_one_hot_enc(df, column_name):
    values = df[column_name].unique()
    unique_value = []
    for value in values:
        if type(value) == str:
            for val in value.split(', '):
                if val not in unique_value:
                    unique_value.append(val)
    df[column_name] = df[column_name].fillna('Unknown')
    for i in unique_value:
        df[i] = df[column_name].apply(lambda x: 1 if i in x else 0)

for i in one_hot_columns:
    str_to_one_hot_enc(df, i)

In [81]:
# Функция для очистки текста description


emoji_pattern = re.compile(
    "["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F700-\U0001F77F"  # alchemical symbols
    u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    u"\U0001FA00-\U0001FA6F"  # Chess Symbols
    u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    u"\U00002702-\U000027B0"  # Dingbats
    u"\U000024C2-\U0001F251" 
    "]+", flags=re.UNICODE)

def clean_text(text):
    if isinstance(text, str):  # Проверяем, является ли текст строкой
        soup = BeautifulSoup(text, "html.parser")
        clean_text = soup.get_text(separator=" ", strip=True) # удаление html
        clean_text = emoji_pattern.sub(r'', clean_text) # удаление emoji
        clean_text = re.sub(r'http\S+', '', clean_text) # удаление ссылок 
        clean_text = re.sub(r'&[a-zA-Z0-9#]+;', '', clean_text) #Удаляет специальные символы
        clean_text = re.sub(r'\d+', '', clean_text) # Удаляет все цифры
        clean_text = re.sub(r'[—®«»]', '', clean_text) # Удаляет некоторые специальные символы
        clean_text = re.sub(r'\s+', ' ', clean_text) # Заменяет последовательности пробелов на один пробел
        clean_text = clean_text.translate(str.maketrans('', '', string.punctuation)) # Удаляет все знаки пунктуации с использованием
        clean_text = clean_text.lower() # Приводит текст к нижнему регистру.
        stop_words_en = set(stopwords.words('english')) # 
        stop_words_ru = set(stopwords.words('russian')) # 
        words = clean_text.split() # 
        clean_text = ' '.join([word for word in words if word not in stop_words_en and word not in stop_words_ru]) # Удаляет стоп-слова на английском и русском языках
        clean_text = re.sub(r'[—®«»※①②③④⑤⑥⑦⑧⑨⑩~――‥‥`“”…ⅱ]', '', clean_text) # Удаляет некоторые специальные символы
        clean_text = re.sub(r'\b\w{2}\b', '', clean_text) # Удаляет отдельно стоящие слова из двух символов
        clean_text = re.sub(r'[^\x00-\x7F]+', '', clean_text) # Удаляет порченные 
        clean_text = re.sub(r'^[\u4E00-\u9FFF\u3400-\u4DBF\u20000-\u2A6DF\u2A700-\u2B73F\u2B740-\u2B81F\u2B820-\u2CEAF\uF900-\uFAFF\u2F800-\u2FA1F]+$', '', clean_text)
        pattern = r'[^\x00-\x7F\u0400-\u04FF\u3040-\u309F\u30A0-\u30FF\uAC00-\uD7AF\u1100-\u11FF\u3130-\u318F]+'
        clean_text = re.sub(pattern, '', clean_text)
        clean_text = re.sub(r'\s+', ' ', clean_text)
        clean_text = re.sub(r'\b\w{2}\b', '', clean_text)
        clean_text = re.sub(r'\s+', ' ', clean_text)
        return clean_text
    else:
        return text

In [82]:
# Функция для извлечения параметров min_req


os_pattern = re.compile(r'(?:ОС|OS):\s*(.*?)<br>')
processor_pattern = re.compile(r'(?:Процессор|Processor):\s*(.*?)<br>')
ram_pattern = re.compile(r'(?:Оперативная память|Memory):\s*(.*?)<br>')
graphics_pattern = re.compile(r'(?:Видеокарта|Graphics):\s*(.*?)<br>')
storage_pattern = re.compile(r'(?:Место на диске|Hard Drive):\s*(.*?)<br>')
sound_pattern = re.compile(r'(?:Звуковая карта|Sound):\s*(.*?)<br>')
vr_pattern = re.compile(r'(?:Поддержка VR):\s*(.*?)<br>')

# Функция для извлечения параметров по заданному регулярному выражению
def extract_param(pattern, text):
    match = re.search(pattern, text)
    if match:
        return match.group(1).strip()
    else:
        return None
    

def extract_params(text):
    if isinstance(text, str):  # Проверяем, что значение является строкой
        os = extract_param(os_pattern, text)
        processor = extract_param(processor_pattern, text)
        ram = extract_param(ram_pattern, text)
        graphics = extract_param(graphics_pattern, text)
        storage = extract_param(storage_pattern, text)
        sound = extract_param(sound_pattern, text)
        vr =  extract_param(vr_pattern, text)
        
        return pd.Series([os, processor, ram, graphics, storage, sound, vr], index=['OS', 'Processor', 'RAM', 'Graphics', 'Storage', 'Sound', 'VR'])
    else:
        return pd.Series([None, None, None, None, None, None, None], index=['OS', 'Processor', 'RAM', 'Graphics', 'Storage', 'Sound', 'VR'])
    

def clean_min_req(text):
    if isinstance(text, str):
        return re.sub(re.compile(r'<.*?>'), '', text).strip()
    else:
        return None
    

In [83]:
def count_words(text):
    if isinstance(text, str):
        return len(text.split())
    else:
        return 0

In [84]:
def preprocess_dataframe(df):

    print("Количество строк до удаления:", len(df))

    # Удаление повторяющихся строк
    df = df.drop_duplicates()


    # Удаление всех нечисловых символов и приводим в норм вид возраст
    df['required_age'] = df['required_age'].str.replace(r'[^\d]', '', regex=True) 
    df['required_age'] = pd.to_numeric(df['required_age'], errors='coerce').fillna(0).astype(int) 
    df['required_age'] = df['required_age'].replace(0, 18)

    # Конвертация в MM YY
    df['release_date'] = df['release_date'].apply(convert_release_date)
    
    # Чистка description
    df['detailed_description'] = df['detailed_description'].apply(clean_text)
    df['short_description'] = df['short_description'].fillna('Unknown')

    

    # Чистка и извлечение параметров min_req
    clean_extracted_params = df['min_req'].apply(extract_params).map(clean_min_req)
    df = pd.concat([df, clean_extracted_params], axis=1)


    # Удаление строк, содержащих "порченные" символы в detailed_description
    corrupted_regex = re.compile(r'[^\x00-\x7F]+')
    df = df[~df['detailed_description'].apply(lambda x: bool(corrupted_regex.search(str(x))))]

    # Удаляем строки где описание состоит только из иероглифов
    chinese_regex = re.compile(r'^[\u4E00-\u9FFF\u3400-\u4DBF\u20000-\u2A6DF\u2A700-\u2B73F\u2B740-\u2B81F\u2B820-\u2CEAF\uF900-\uFAFF\u2F800-\u2FA1F]+$')
    df = df[~df['detailed_description'].apply(lambda x: bool(chinese_regex.match(x)))]
    # df = df[~df['name'].apply(lambda x: bool(chinese_regex.match(x)))]
    df = df[~df['short_description'].apply(lambda x: bool(chinese_regex.match(x)))]

    # pattern = r'[^\x00-\x7F\u0400-\u04FF\u3040-\u309F\u30A0-\u30FF\uAC00-\uD7AF\u1100-\u11FF\u3130-\u318F]+'
    # df['name'] = df['name'].apply(lambda x: re.sub(pattern, '', x))


    # Удаление коротких описаний (длиной менее 20 символов)
    df['word_count'] = df['detailed_description'].apply(count_words)
    df = df[df['word_count'] >= 20]

    # Дропаем колонки 
    columns_to_drop = ['Unnamed: 0', 'min_req', 'rec_req', 'about_the_game', 'word_count' ]  
    df = df.drop(columns_to_drop, axis=1, errors='ignore')
    print("Количество строк после удаления:", len(df))

    return df

df = preprocess_dataframe(df)
df.head()

Количество строк до удаления: 139256


/var/folders/vb/3hggmwrs7794d_7wkqtk7xtm0000gn/T/ipykernel_45160/1359640670.py:21: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


Количество строк после удаления: 99910


,steam_appid,type,name,required_age,platforms,categories,genres,detailed_description,short_description,is_free,developers,publishers,release_date,metacritic,windows,mac,linux,Для одного игрока,Для нескольких игроков,Против игроков,Против игроков (по сети),Достижения Steam,Отслеживание контроллеров,Только для VR,Коллекционные карточки,Статистика,Family Library Sharing,Дополнительный контент,Мастерская Steam,Контроллер (частично),Таблицы лидеров Steam,Включает редактор уровней,Steam Cloud,Контроллер (полностью),Remote Play на телевизоре,Кооперативная игра,Кроссплатформенная игра,Кооператив (общий экран),Общий экран,Remote Play Together,Поддержка VR,Кооператив (по сети),Имеются субтитры,MMO,Против игроков (общий экран),Покупки внутри приложения,Виртуальная реальность,Предметы для SteamVR,Remote Play на телефоне,Remote Play на планшете,Имеется эффект HDR,Включает Source SDK,Имеется античит Valve,Имеются комментарии,Против игроков (LAN),Уведомления о новых ходах,Кооператив (LAN),Высококачественное аудио,Mods,Мод (требуется HL2),Экшены,Инди,Симуляторы,Ранний доступ,Стратегии,Казуальные игры,Гонки,Приключенческие игры,Бесплатно,Ролевые игры,Спортивные игры,Многопользовательские игры,Насилие,Мясо,Сексуальный контент,Нагота,Веб-разработка,Дизайн и иллюстрация,Утилиты,Анимация и моделирование,Работа со звуком,Образование,Обучение работе с ПО,Создание видео,Обработка фото,Разработка игр,Бухгалтерия,Фильм,Короткометражный фильм,OS,Processor,RAM,Graphics,Storage,Sound,VR
0,612250,game,Eastwood VR,18,windows,"Для одного игрока, Для нескольких игроков, Про...","Экшены, Инди, Симуляторы, Ранний доступ",trading cards available eastwoodvr wild west d...,EastwoodVR - is a wild west VR duel simulation...,False,RAV3 Interactive,RAV3 Interactive,May 2017,Unknown,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None,Intel Core i5 or higher,3000 MB ОЗУ,"GeForce GTX 960, R9 285 or higher",700 MB,None,None
1,612260,dlc,Fernbus Simulator - Anniversary Repaint Package,18,windows,"Для одного игрока, Дополнительный контент, Дос...",Симуляторы,flixbus fernbus simulator dlc man lions coach...,В этом пакете вы получите 4 абсолютно новых и ...,False,TML-Studios,Aerosoft GmbH,Jun 2017,Unknown,1,0,0,1,0,0,0,1,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7/8/8.1/10 (64bit only),Intel Core i5 Processor or similar with at lea...,6 GB ОЗУ,Nvidia GeForce GTX 560 or similar AMD Radeon (...,100 MB,None,None
3,612300,game,Sudden Strike Gold,18,windows,"Для одного игрока, Включает редактор уровней, ...",Стратегии,sudden strike notice rerelease classic game ev...,"Set in World War 2, Sudden Strike offers revol...",False,Fireglow,Kalypso Media Digital,May 2017,Unknown,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None,Intel or AMD 2Ghz Dual-Core CPU,1 GB ОЗУ,"DirectX 9.0c hardware compatible, 256 MB RAM",None,None,None
4,612310,game,GORB,18,"windows, mac, linux","Для одного игрока, Достижения Steam, Steam Clo...","Казуальные игры, Инди, Симуляторы, Стратегии",gorb physics based puzzle game simple mechanic...,"Rid the world of Red and Orange shapes, while ...",False,Jon Gallant,jgallant,Apr 2017,Unknown,1,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Windows,Pentium Dual Core 2.4Ghz,256 MB ОЗУ,DirectX 9.0 Capable Graphics Card,200 MB,None,None
5,612370,game,PAKO 2,18,"windows, mac, linux","Для одного игрока, Достижения Steam, Контролле...","Экшены, Казуальные игры, Инди, Гонки",get get get paid explore vast cities getaway d...,Arcade drive-by shooting action with heist gam...,False,Tree Men Games,Tree Men Games,Nov 2017,Unknown,1,1,1,1,0,0,0,1,0,0,1,1,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,

# Обучение модели

## сокращение df

In [85]:
cropped_df = df[['steam_appid', 'name', 'type', 'detailed_description']]

## SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [97]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
import faiss
from tqdm import tqdm, trange
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='tqdm')

In [87]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
descriptions = cropped_df['detailed_description'].tolist()
description_embeddings = model.encode(descriptions, convert_to_tensor=True)

In [99]:
description_embeddings_np = description_embeddings.cpu().numpy()
index = faiss.IndexFlatL2(description_embeddings_np.shape[1])
index.add(description_embeddings_np)

In [100]:
# Пользовательский текст
user_input = "indie game with open world"
user_embedding = model.encode(user_input, convert_to_tensor=True)

In [101]:
# Вычисление косинусного сходства
cosine_scores = util.pytorch_cos_sim(user_embedding, description_embeddings)

# Получение индексов наиболее похожих описаний
top_k = 10
top_results = torch.topk(cosine_scores, k=top_k)

# Преобразование индексов в целые числа
indices = top_results.indices[0].tolist()

# Вывод рекомендованных игр и DLC
print("Top recommendations:")
for idx, score in zip(indices, top_results.values[0]):
    print(f"Name:{df['name'].iloc[idx]} \ntype: {df['type'].iloc[idx]}\n(steam_appid: {df['steam_appid'].iloc[idx]})\nScore: {score.item():.4f}\n Description:{df['short_description'].iloc[idx]}\nDetailed descr:{df['detailed_description'].iloc[idx]}\n{'-----'*10}")

Top recommendations:
Name:Open World Game: the Open World Game 
type: game
(steam_appid: 1144110)
Score: 0.6944
 Description:Open World Game: the Open World Game is the purest open world game experience. Enjoy simplified mechanics, minimal graphics, and an extremely short main story; all so you can get to work removing every icon from the map with the least resistance.
Detailed descr:open world game open world game purest open world game experience broke open world games core dont deal gorgeous environments distracting stellar voice acting performances drawing even prolonged epic main quest constantly weighing ignoring lieu collecting every card mini game pure unadulterated open world game game pickup icons scattered across open world epic story told journal entries ingame achievements fishing branch skill tree perfectly crafted make decision making easy yeah satire
--------------------------------------------------
Name:Mayhems World 
type: game
(steam_appid: 2424710)
Score: 0.6694
 D

In [104]:
index_file = '/Users/mossyhead/ds_bootcamp/GameExplorer/model/description_embeddings.index'
model_file = '/Users/mossyhead/ds_bootcamp/GameExplorer/model/paraphrase-multilingual-MiniLM-L12-v2.pth'
embeddings_file = '/Users/mossyhead/ds_bootcamp/GameExplorer/model/description_embeddings.npy'

# FAISS index
faiss.write_index(index, index_file)

# SentenceTransformer model
torch.save(model.state_dict(), model_file)

#  embeddings 
np.save(embeddings_file, description_embeddings_np)

In [103]:
df.to_csv('/Users/mossyhead/ds_bootcamp/GameExplorer/steam_data/csv_files/cleaned_data.csv', index=False)
cropped_df.to_csv('/Users/mossyhead/ds_bootcamp/GameExplorer/steam_data/csv_files/cropped_data.csv', index=False)